<div align="center">
    <img src="./OperationNextNest.png" width="400"/><br>
</div>

---

<div align="center">

## 🏡 Operation Next Nest | Part 3 | Neighborhood Mapping in R

</div>

---

### Maps, Scenarios & Smarter Decisions with R

Welcome to **Part 3** of *Operation Next Nest* — where our analysis finally comes to life on the map! 🌍✨  

Let’s quickly recap our adventure so far:

---

### 🧩 Part 1 | Data Engineering in Python
We:
- Pulled **ACS Census** data for Wake & Durham block groups  
- Calculated **commute distances + travel times** to the SAS Campus  
- Pulled **amenities** (parks, groceries, bike trails) from OpenStreetMap  
- Exported a clean dataset for modeling ✔️  

---

### 🚀 Part 2 | Optimization in SAS
We:
- Standardized key variables like:
  - 🚗 Commute time  
  - 🏠 Housing affordability  
  - 💼 Income + employment strength  
  - 👶 Kid-friendliness  
  - 🌳 Local amenities  
- Built a **weighted score** for every neighborhood
- Ran multiple optimization scenarios using `PROC OPTMODEL` 🎯
- Exported the results to CSV 📤

---

### 🎯 Now: Pretty Maps in R!

Turn optimized results into **visual decision support tools**:

| Objective | Technique |
|----------|-----------|
| Join SAS scenario results with maps | `sf`, `tigris` |
| Visualize weighted scores across the Triangle | `leaflet` choropleths |
| Compare scenario outcomes on the same map | jittered overlays |
| Identify **robust picks** chosen multiple times | stability shading |

---

### Why this matters 🤔

Data is most powerful when it’s **understandable** and **actionable**.  
Today, we’ll see how different priorities change the story about:

> **Where should we look for our next home?**

Ready to unlock some insights? 🔑  
Let’s map our way to smarter decisions → ⬇️



---

## 📂 Start at the top: import packages and data
### (Just a quick pit stop before mapping!)

In **Part 2**, we ran our optimization model under several different:
- priorities ⚖️  
- neighborhood preferences 🏘️  
- family values 👨‍👩‍👦  

We exported results for each scenario as a **CSV** — which keeps our `GEOID` identifiers nice & clean for joining to a map. ✔️

---

### 🔎 What’s inside each scenario result?

For **every** Census block group in Wake + Durham, we have:

**Model Inputs**
- Travel time to SAS campus 🚗
- Median home value 🏠
- Median income 💵
- Kid/family friendliness indicators 👶
- Amenity access 🌳🍎🚲

**Model Outputs**
- `weighted_score` — overall “goodness”
- `chosen_flag` — was this block group selected as a **top 10**?
- Which scenario selected it 🏆

All four scenarios are stacked together into one convenient table:

| Scenario | Priority Focus |
|---------|----------------|
| Base | Balanced 🌈 |
| CommuteFirst | Faster commute ⏱️ |
| KidFriendly | Families first 👶 |
| AmenityHeavy | Walkability + parks 🌳 |

---

### 🎯 Goal of this step

Load **all** scenario results into R:

- Keep `GEOID` as a **character string** (critical!)
- Add a `scenario` label to each row
- Do a quick health check 🔍

Then — on to mapping! 🗺️🔥

Ready? Let’s import → ⬇️


In [1]:
# Install once if needed:
install.packages(c("readr", "dplyr", "purrr", "ggplot2", "sf", "tigris", "leaflet"))

library(readr)
library(dplyr)
library(purrr)
library(ggplot2)
library(sf)
library(tigris)
library(leaflet)

Installing packages into ‘/workspaces/myfolder/.user-R-packages’
(as ‘lib’ is unspecified)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.7.2, GDAL 3.0.4, PROJ 6.3.2; sf_use_s2() is TRUE

To enable caching of data, set `options(tigris_use_cache = TRUE)`
in your R script or .Rprofile.



In [2]:
options(tigris_use_cache = TRUE)

data_path <- "/workspaces/myfolder/OperationNextNest/Data"

scenario_names <- c("Base", "CommuteFirst", "KidFriendly", "AmenityHeavy")

In [3]:
# Helper: read one scenario CSV and tag it
read_scenario_csv <- function(scen) {
  file <- file.path(data_path, paste0("onn_result_", scen, ".csv"))
  if (!file.exists(file)) {
    warning("File not found for scenario: ", scen, " at ", file)
    return(NULL)
  }

  # Make sure GEOID comes in as character, not numeric
  read_csv(
    file,
    show_col_types = FALSE,
    col_types = cols(
      GEOID = col_character(),
      .default = col_guess()
    )
  ) %>%
    mutate(scenario = scen)
}

# Read and stack all scenarios
onn_all <- map_dfr(scenario_names, read_scenario_csv)

dplyr::glimpse(onn_all)


Rows: 2,964
Columns: 17
$ GEOID                <chr> "371830525061", "371830535171", "371830525042", "…
$ travel_time_min      <dbl> 0.14276047, 0.07039031, 0.11555899, 0.17021749, 0…
$ distance_miles       <dbl> 0.14276047, 0.07039031, 0.11555899, 0.17021749, 0…
$ median_value         <dbl> 0.3399741, 0.0000000, 0.2626019, 0.3097665, 0.166…
$ median_income        <dbl> 0.9997060, 0.9997079, 0.9997526, 0.9998238, 0.999…
$ bg_id                <dbl> 704, 389, 191, 522, 385, 235, 19, 203, 549, 15, 2…
$ commute_score        <dbl> 0.8572395, 0.9296097, 0.8844410, 0.8297825, 0.666…
$ housing_afford_score <dbl> 0.6600259, 1.0000000, 0.7373981, 0.6902335, 0.833…
$ income_score         <dbl> 0.9997060, 0.9997079, 0.9997526, 0.9998238, 0.999…
$ kids_score           <dbl> 0.4750535, 0.3912543, 0.6182109, 0.7079305, 0.730…
$ amenity_score        <dbl> 1.0000000, 0.6666667, 0.6666667, 0.6666667, 0.666…
$ weighted_score       <dbl> 0.7650490, 0.7635201, 0.7564512, 0.7557907, 0.749…
$ chosen_flag   

In [4]:
# Keep only Base scenario for first, simple map
onn_base <- onn_all %>% filter(scenario == "Base")

# Quick sanity check: chosen vs not chosen in Base
onn_base %>%
  count(chosen_flag) %>%
  mutate(pct = round(100 * n / sum(n), 1))


chosen_flag,n,pct
<dbl>,<int>,<dbl>
1,10,1.3
NA,731,98.7


***

## 🗺️ Bring on the Geography!
### Adding Block Group Boundaries for Real-World Context

Our optimized results tell us **which** neighborhoods scored well…  
…but **where are they**? 🤷‍♂️

Time to connect the dots — literally!

---

### 🧱 What we need:

A map of all:
- 🏘️ Census Block Groups in **Wake** and **Durham**
- With **polygon geometries** so we can draw them

We’ll grab them from the **TIGER/Line** dataset via the `tigris` package:

| Column | Description |
|--------|-------------|
| `GEOID` | Unique identifier for each block group (our join key!) |
| `COUNTYFP` | County code → Wake = `183`, Durham = `063` |
| `geometry` | Shape of each neighborhood area |

---

### 🔗 Join Spatial Shapes to Optimization Results

Once we have the shapes, we’ll:

1. Convert geometries to **WGS84 coordinates** (lat/lon)  
2. Filter only Wake + Durham counties  
3. Join by **GEOID** to pull in the scores + flags from SAS  
4. Do a sanity check 🧪  
   - We expect lots of **numbers**, **not NA**  
   - If it’s all NAs → there’s a mismatch in GEOIDs 😬

➡️ When this step works, we’ll have a **fully spatial dataset** ready to map!

---

Let’s download some shapes → ⬇️🌍


In [5]:
# 1. Get NC block groups (simplified boundaries for speed)
nc_bg <- block_groups(state = "NC", year = 2023, cb = TRUE) %>%
  mutate(GEOID = as.character(GEOID))

# 2. Filter to Wake (183) + Durham (063)
wake_durham_bg <- nc_bg %>%
  filter(COUNTYFP %in% c("183", "063"))

# 3. Join Base scenario to geometry
onn_base_sf <- wake_durham_bg %>%
  left_join(onn_base, by = "GEOID")

# Quick check: do we now have non-missing weighted_score?
summary(onn_base_sf$weighted_score)


  |======================================================================| 100%


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
 0.2510  0.4937  0.5667  0.5585  0.6298  0.7650      94 


---

## 🗺️ Interactive Map of Neighborhood “Goodness”
### Weighted Score + Real Street Map Views

It’s time for our **first map reveal**… and it’s an interactive one! 🎉

We’ll use the **`leaflet`** package — the same tech that powers many real estate maps 🏘️  
It gives us:

✔ Pan + zoom + explore the city  
✔ Rich base maps (labels, streets, parks, context!)  
✔ Popups with score + travel + housing info  
✔ Outlined optimized neighborhoods for the chosen scenario 🏆

---

### What are we mapping?

Every neighborhood in Wake & Durham is shaded by:

> **`weighted_score` — a combined measure of commute + affordability + income + amenities + kids**

The “better” the score →
the **brighter** the neighborhood shines 🌟

We’ll start with the **Base** scenario to get our bearings…

➡️ Ready to see the first result of all our hard modeling effort?  
Let’s visualize! ⬇️


In [6]:
# 1. Ensure geometry is in WGS84 (lat/lon) for web maps
onn_base_sf_wake_dur <- st_transform(onn_base_sf, 4326)

# 2. Office location: SAS Campus
office_sf <- st_sf(
  name = "SAS Campus",
  geometry = st_sfc(
    st_point(c(-78.74939336654782, 35.81563890659871)), # lon, lat
    crs = 4326
  )
)

# 3. Build palette using non-missing weighted scores
valid_scores <- onn_base_sf_wake_dur$weighted_score[!is.na(onn_base_sf_wake_dur$weighted_score)]

score_pal <- colorNumeric(
  palette = "plasma",
  domain  = valid_scores,
  na.color = "transparent"
)

# 4. Leaflet map
leaflet() %>%
  addProviderTiles(providers$CartoDB.Positron) %>%

  # Fill all block groups by weighted_score
  addPolygons(
    data = onn_base_sf_wake_dur,
    fillColor   = ~score_pal(weighted_score),
    fillOpacity = 0.7,
    color       = "#aaaaaa",
    weight      = 0.3,
    opacity     = 0.6,
    smoothFactor = 0.2,
    highlightOptions = highlightOptions(
      weight = 1,
      color = "#333333",
      bringToFront = TRUE
    ),
    popup = ~paste0(
      "<b>GEOID:</b> ", GEOID, "<br>",
      "<b>Weighted score:</b> ", round(weighted_score, 3), "<br>",
      "<b>Travel time (min):</b> ", round(travel_time_min, 1), "<br>",
      "<b>Median value:</b> $", format(round(median_value, 0), big.mark = ",")
    )
  ) %>%

  # Outline chosen neighborhoods for Base scenario
  addPolygons(
    data = subset(onn_base_sf_wake_dur, chosen_flag == 1),
    fillColor   = NA,
    fillOpacity = 0,
    color       = "black",
    weight      = 2,
    opacity     = 1
  ) %>%

  # Add SAS Campus marker
  addCircleMarkers(
    data = office_sf,
    radius = 7,
    stroke = TRUE,
    weight = 2,
    color = "black",
    fillColor = "yellow",
    fillOpacity = 1,
    popup = ~name,
    label = ~name
  ) %>%

  # Legend
  addLegend(
    position = "bottomright",
    pal      = score_pal,
    values   = valid_scores,
    title    = "Weighted Score",
    opacity  = 1
  ) %>%

  # Center between Raleigh & Durham
  setView(
    lng = -78.85,
    lat = 35.95,
    zoom = 10
  )


HTML widgets cannot be represented in plain text (need html)


---

## 🔁 Multi-Scenario Exploration
### How Do Different Priorities Change the “Best” Neighborhoods?

In Part 2, we didn’t stop at just one set of preferences…  
we tried out **multiple** “what matters most?” scenarios! 🎯

Each scenario reflects a **different definition of “best place to live”**:

| Scenario | Priority Focus | Emoji Vibe |
|----------|----------------|------------|
| Base | Balanced values | 🌈 |
| CommuteFirst | Closer to SAS Campus | 🚗💨 |
| KidFriendly | Best for young families | 👶🧸 |
| AmenityHeavy | Parks + shops + trails | 🌳🍕🚲 |

---

### Why this is important 💡

Different people want different things —  
and **analytics should support human preferences**, not replace them.

So we ask:

> 💬 *“Which neighborhoods stay awesome under many scenarios?”*

Those become our **robust picks** ✔️  
Great candidates for a house hunt!

---

### Next step:
We’ll filter the dataset to **just the chosen neighborhoods** (the Top 10 per scenario) ✔️

Then, we’ll **visualize**:

- Where each scenario points us 🔍
- Where scenarios disagree 😬
- Where all signs point to the same cluster 😍

⬇️ Buckle up — scenario comparison coming right up!


In [7]:
library(dplyr)
library(sf)
library(leaflet)

# 1. Start from the all-scenario results
#    (onn_all was built earlier from CSVs)
#    Keep only chosen neighborhoods
chosen_all <- onn_all %>%
  filter(chosen_flag == 1)

# 2. Make sure geometry is in WGS84 (lat/lon) for web mapping
wake_durham_bg_4326 <- st_transform(wake_durham_bg, 4326)

# 3. Join chosen neighborhoods to geometry
chosen_sf <- wake_durham_bg_4326 %>%
  inner_join(chosen_all, by = "GEOID")

# 4. Get centroids and extract coordinates
centroids <- suppressWarnings(st_centroid(chosen_sf))  # warning about attributes is OK
coords <- st_coordinates(centroids)

chosen_pts <- centroids %>%
  st_drop_geometry() %>%
  bind_cols(as.data.frame(coords)) %>%
  rename(lon = X, lat = Y)

# 5. Jitter points around each centroid so multi-scenario picks don't overlap
scenario_levels <- unique(chosen_pts$scenario)
scenario_map <- setNames(seq_along(scenario_levels), scenario_levels)

chosen_pts <- chosen_pts %>%
  mutate(
    scen_index = scenario_map[scenario],
    radius = 0.003,  # adjust ring size as needed
    angle = case_when(
      scen_index == 1 ~ 0,
      scen_index == 2 ~ pi / 2,
      scen_index == 3 ~ pi,
      scen_index == 4 ~ 3 * pi / 2,
      TRUE            ~ 0
    ),
    lon_jitter = lon + radius * cos(angle),
    lat_jitter = lat + radius * sin(angle)
  )

# 6. SAS Campus marker
office_sf <- st_sf(
  name = "SAS Campus",
  geometry = st_sfc(
    st_point(c(-78.74939336654782, 35.81563890659871)),  # lon, lat
    crs = 4326
  )
)

# 7. Color palette by scenario
scenario_pal <- colorFactor(
  palette = "Set1",
  domain  = scenario_levels
)

# 8. Leaflet multi-scenario map
leaflet() %>%
  addProviderTiles(providers$CartoDB.Positron) %>%

  # Base polygons (all block groups for context)
  addPolygons(
    data = wake_durham_bg_4326,
    stroke = FALSE,
    fillColor = "grey95",
    fillOpacity = 1
  ) %>%

  # Jittered chosen neighborhoods, colored by scenario
  addCircleMarkers(
    data = chosen_pts,
    lng = ~lon_jitter,
    lat = ~lat_jitter,
    radius = 5,
    stroke = TRUE,
    weight = 1,
    color = "black",
    fillColor = ~scenario_pal(scenario),
    fillOpacity = 0.9,
    popup = ~paste0(
      "<b>Scenario:</b> ", scenario, "<br>",
      "<b>GEOID:</b> ", GEOID, "<br>",
      "<b>Weighted score:</b> ", round(weighted_score, 3), "<br>",
      "<b>Travel time (min):</b> ", round(travel_time_min, 1), "<br>",
      "<b>Median value:</b> $", format(round(median_value, 0), big.mark = ",")
    )
  ) %>%

  # SAS Campus marker
  addCircleMarkers(
    data = office_sf,
    radius = 7,
    stroke = TRUE,
    weight = 2,
    color = "black",
    fillColor = "yellow",
    fillOpacity = 1,
    popup = ~name,
    label = ~name
  ) %>%

  # Legend for scenarios
  addLegend(
    position = "bottomright",
    pal      = scenario_pal,
    values   = scenario_levels,
    title    = "Scenario",
    opacity  = 1
  ) %>%

  # Region view
  setView(
    lng = -78.85,
    lat = 35.95,
    zoom = 10
  )


HTML widgets cannot be represented in plain text (need html)

## 🎯 Notes on the approach above: Visualizing Overlapping Choices
### (Why we need jittering to see the full picture!)

When we map **all scenarios at once**, there’s a catch:  
many scenarios select the **same** top block groups. 🤝

If we plotted their centroid points directly:  
> They’d **perfectly overlap** →  
> we’d only see **1 dot instead of 3 or 4** ❌

And that would hide a *very cool insight*!

---

### ✨ Jitter to the rescue!

We nudge each point **slightly outward** from the centroid —  
like petals around a flower 🌸

So now we can clearly see:

| Scenario shows up here? | Visible on map? |
|------------------------|----------------|
| Base only? | ✔ Single dot |
| 2–3 scenarios? | ✔ A small cluster |
| All 4 scenarios?! | 🌟 A “super-star” neighborhood |

---

### What this reveals 🧠

Neighborhoods that:
- Appear **once** → scenario-specific “specialists”
- Appear **multiple times** → robust candidates worth exploring IRL 🏠✨

This is where visual analytics turns into
**decision intelligence** — not just data on a screen.

---

🎨 So, the above map is **multi-scenario** where →  
Jitter enabled → clarity unlocked 🔓

And we can see which places keep rising to the top 👀⬇️


---

## 🏆 Stability Map — Neighborhood “Hall of Fame”
### How many scenarios select each block group?

This is the **grand reveal** 🎬  
Now that we’ve mapped each scenario individually…  
let’s look at **agreement** across all scenarios.

---

### 🌟 What does “stability” mean here?

How many of our four scenarios chose the **same** block group?

| # of Scenarios Selecting a BG | What It Means |
|------------------------------|---------------|
| 0 | Good, but not a top-10 anywhere 🤷‍♂️ |
| 1 | A specialist — only strong in one priority |
| 2–3 | Versatile — strong across multiple preferences 💪 |
| 4 | ⭐⭐ **Consensus Rock Star** ⭐⭐<br/>A neighborhood loved by ALL scenarios |

These are the areas that **everyone agrees** deserve a closer look in real life.  
(Like, *“yep, we all want to live there”* ✔️)

---

### 🔥 Why this map is powerful

This visualization helps answer:

> *“Which neighborhoods are consistently great, no matter how we weigh the priorities?”*

It turns our optimization into **smart decision support**:
- Shows where to focus the house hunt 🎯
- Highlights resilient choices under changing assumptions
- Builds trust in the model (not just 1 scenario’s opinion)

---

### Ready to see the “MVPs of the Triangle”? 🏡🙌  
Stability choropleth → coming up next ⬇️


In [8]:
library(leaflet)

# 1. How many scenarios chose each GEOID?
stability <- chosen_all %>%
  count(GEOID, name = "n_scenarios")

# 2. Join to geometry (WGS84 version)
stability_sf <- wake_durham_bg_4326 %>%
  left_join(stability, by = "GEOID") %>%
  mutate(n_scenarios = ifelse(is.na(n_scenarios), 0L, n_scenarios))

# 3. Palette for # of scenarios
max_scen <- length(scenario_names)
stab_pal <- colorNumeric(
  palette = "plasma",
  domain  = 0:max_scen,
  na.color = "transparent"
)

# 4. Leaflet stability map
leaflet() %>%
  addProviderTiles(providers$CartoDB.Positron) %>%
  addPolygons(
    data = stability_sf,
    fillColor   = ~stab_pal(n_scenarios),
    fillOpacity = 0.8,
    color       = "#aaaaaa",
    weight      = 0.3,
    opacity     = 0.6,
    popup = ~paste0(
      "<b>GEOID:</b> ", GEOID, "<br>",
      "<b># of scenarios selecting this BG:</b> ", n_scenarios
    )
  ) %>%
  addLegend(
    position = "bottomright",
    pal      = stab_pal,
    values   = 0:max_scen,
    title    = "# of scenarios\nselecting BG",
    opacity  = 1
  ) %>%
  setView(
    lng = -78.85,
    lat = 35.95,
    zoom = 10
  )


HTML widgets cannot be represented in plain text (need html)

## 🎯 Wrap-Up: What Did We Learn?

We asked a real question:

> **“Where should we look for our next home?”**

And we used a full analytics workflow to help answer it:

| Phase | Tool | Purpose |
|------|------|---------|
| Part 1 | Python | Pull data from Census, calculate commute time & amenities |
| Part 2 | SAS Optimization | Score neighborhoods and pick optimal solutions |
| Part 3 | R Visualization | Map our results and compare scenarios |

Along the way, we learned:

- Data lives in **different places and formats**
- Geography matters — we needed **lat/long** for the world to make sense
- A single answer isn’t enough — different priorities lead to different choices
- **Robust picks** show up across multiple scenarios

This is analytics for **decision support** — not “one right answer” but **informed tradeoffs**.

---

### 🤔 Reflection Prompts (quick discussion)

• Which scenario aligns the most with *your* values?  
• How did the mapping change your interpretation of the results?  
• What would you want to add next time (schools? crime? cost of living?)  
• At what point might “the model” conflict with “your gut”?  

There’s no magic algorithm to define “home” — but we can absolutely make that search **smarter**.
